# CNN Training

This notebook builds and trains a basic CNN model.

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB5, EfficientNetB7, VGG19
from tensorflow.keras.applications.efficientnet import decode_predictions

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Softmax, BatchNormalization, Input
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, LambdaCallback
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import schedules

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

import util

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_img_dir='../../data/train_images'
test_img_dir='../../data/test_images/images'
train_csv_path = '../../data/train.csv'
test_csv_path = '../../data/test.csv'

In [4]:
train = pd.read_csv(train_csv_path)
test = pd.read_csv(test_csv_path)

In [5]:
tmp = train.groupby(['label_group'])['posting_id'].unique().to_dict()
train['matches'] = train['label_group'].map(tmp)
train['matches'] = train['matches'].apply(lambda x: ' '.join(x))

In [6]:
len(train)

34250

In [7]:
epochs = 40
batch_size = 8
img_width = 224
img_height = 224

In [8]:
train_size = 34250

In [9]:
# 0:20:00 to load 224*34250
start = datetime.now()
X_train = util.load_img_RGB(train_img_dir,train.image[:train_size], img_width, img_height)
X_train /= 255
print(datetime.now()-start)

0:05:02.007787


In [49]:
y_train = np.array(train.label_group[:train_size])
y_train, label_encoder = util.prepare_labels(y_train)

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [50]:
num_class = np.unique(train.label_group[:train_size]).size

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2, random_state=3162)

In [15]:
def tf_dataset(images,labels,mode):
    ds = tf.data.Dataset.from_tensor_slices((images,labels))
    if mode == 'train':
        ds = ds.shuffle(2000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [16]:
ds_train = tf_dataset(X_train,y_train,'train')
ds_val = tf_dataset(X_val,y_val,'validation')

-------

In [16]:
input_layer = tf.keras.layers.Input(shape = (img_width,img_height, 3))
x = EfficientNetB5(include_top = False, weights = 'imagenet')(input_layer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
output = tf.keras.layers.Dense(num_class, activation = 'softmax')(x)
model = tf.keras.models.Model(inputs = [input_layer], outputs = [output])

In [18]:
adam = optimizers.Adam(0.0001)
model.compile(loss = "categorical_crossentropy", 
               optimizer = adam, 
               metrics=["accuracy"])

In [19]:
scheduler_exp = schedules.ExponentialDecay(0.0001, decay_steps=100000, decay_rate=1e-6)
adam = optimizers.Adam(scheduler_exp,clipnorm=0.1)

In [21]:
print_weights = LambdaCallback(on_epoch_begin=lambda batch, logs: print(model.layers[7].get_weights()))

In [22]:
callbacks = [EarlyStopping(patience=20,restore_best_weights=True)]
#             print_weights]

In [23]:
epochs=40

In [24]:
start = datetime.now()
history = model.fit(ds_train, 
                    batch_size=batch_size, epochs=epochs,
                    verbose=1,
                    shuffle=True, validation_data=ds_val,
                    callbacks=callbacks)
print(datetime.now()-start)

Epoch 1/40
3425/3425 [==============================] - 699s 177ms/step - loss: 9.2179 - accuracy: 0.0046 - val_loss: 9.0845 - val_accuracy: 0.0104
Epoch 2/40
3425/3425 [==============================] - 603s 176ms/step - loss: 8.5588 - accuracy: 0.0164 - val_loss: 8.8696 - val_accuracy: 0.0223
Epoch 3/40
3425/3425 [==============================] - 604s 176ms/step - loss: 7.9963 - accuracy: 0.0329 - val_loss: 8.9261 - val_accuracy: 0.0387
Epoch 4/40
3425/3425 [==============================] - 603s 176ms/step - loss: 7.4017 - accuracy: 0.0542 - val_loss: 8.8399 - val_accuracy: 0.0531
Epoch 5/40
3425/3425 [==============================] - 602s 176ms/step - loss: 6.9080 - accuracy: 0.0789 - val_loss: 8.9990 - val_accuracy: 0.0778
Epoch 6/40
3425/3425 [==============================] - 603s 176ms/step - loss: 6.4239 - accuracy: 0.1074 - val_loss: 9.3242 - val_accuracy: 0.0945
Epoch 7/40
3425/3425 [==============================] - 603s 176ms/step - loss: 5.9799 - accuracy: 0.1353 - val_

In [30]:
model.save('./trained/my_model_h5.h5')

/home/renyc/Anaconda/anaconda3/envs/python3.8.3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [26]:
hist_df = pd.DataFrame(history.history)
hist_df.to_json('./trained/training_hist.json')